In [7]:
import arrayfire as af
module_input_1 = af.array.read_array("OUTPUT.arr", key="trf_input_0").to_ndarray()
module_output_1 = af.array.read_array("OUTPUT.arr", key="trf_output_0").to_ndarray()
module_input_2 = af.array.read_array("OUTPUT.arr", key="trf_input_1").to_ndarray()
module_output_2 = af.array.read_array("OUTPUT.arr", key="trf_output_2").to_ndarray()
def valueprint(arr, key=None):
    print(key, arr.shape, arr.sum())

valueprint(module_input_1)
valueprint(module_output_1)
valueprint(module_input_2)
valueprint(module_output_2)

None (1536, 321) 330.57333
None (1536, 321) 14736.917
None (1536, 321) 14736.917
None (1536, 321) 17337.0


In [8]:
# testing after fixing the model & weights
import torch
from transformers import MCTCForCTC, MCTCProcessor, MCTCConfig
config = MCTCConfig()
model = MCTCForCTC(config)
model.load_state_dict(torch.load("./ported_pytorch_model.bin"))
model.eval()

MCTCForCTC(
  (mctc): MCTCModel(
    (encoder): MCTCEncoder(
      (layer_norm): MCTCLayerNorm()
      (conv): Conv1dSubsampler(
        (dropout): Dropout(p=0.3, inplace=False)
        (conv_layers): ModuleList(
          (0): Conv1d(80, 3072, kernel_size=(7,), stride=(3,), padding=valid)
        )
      )
      (layers): ModuleList(
        (0): MCTCLayer(
          (intermediate): MCTCIntermediate(
            (dense): Linear(in_features=1536, out_features=6144, bias=False)
            (intermediate_act_fn): ReLU()
          )
          (attention): MCTCAttention(
            (self): MCTCSelfAttention(
              (query): Linear(in_features=1536, out_features=1536, bias=False)
              (key): Linear(in_features=1536, out_features=1536, bias=False)
              (value): Linear(in_features=1536, out_features=1536, bias=False)
              (dropout): Dropout(p=0.3, inplace=False)
              (distance_embedding): Embedding(1839, 384)
            )
            (output): MCTC

In [9]:
model = model.eval()
first_layer = model.mctc.encoder.layers[0]
second_layer = model.mctc.encoder.layers[1]

In [10]:
model = model.eval()
our_output = torch.Tensor(module_input_1).unsqueeze(0).transpose(1, 2)
valueprint(our_output, "input")
print()
our_output = first_layer(our_output)[0]
valueprint(our_output, "our_output_1")
valueprint(module_output_1, "module_output_1")
print()
our_output = second_layer(our_output)[0]
valueprint(our_output, "our_output_2")
valueprint(module_output_2, "module_output_2")

input torch.Size([1, 321, 1536]) tensor(330.5734)

our_output_1 torch.Size([1, 321, 1536]) tensor(14504.3711, grad_fn=<SumBackward0>)
module_output_1 (1536, 321) 14736.917

our_output_2 torch.Size([1, 321, 1536]) tensor(12090.7656, grad_fn=<SumBackward0>)
module_output_2 (1536, 321) 17337.0


In [2]:
# lets take a deeper look.
import arrayfire as af
possib_keys = ["selfAttention_input", "selfAttention_posEmb", "selfAttention_mask", "selfAttention_result", "selfAttention_result_2",
"layer_x", "layer_h_preln", "layer_h", "MHAttention_query", "MHAttention_key", "MHAttention_value", "MHAttention_q", "MHAttention_k", "MHAttention_v",
"MHAttention_scores", "MHAttention_pscores", "MHAttention_pscores_scores", "MHAttention_mask", "MHAttention_mask_scores", "MHAttention_padMaskTile", "MHAttention_padMaskTile_scores",
"MHAttention_result", "MHAttention_result_attn", "MHAttention_result_result", "MHAttention_result_result_2"]
trf_outputs = {}
for key in possib_keys:
    try:
        value = af.array.read_array("OUTPUT_TRF.arr", key=key).to_ndarray()
        trf_outputs[key] = value
    except:
        print("FAIL:", key)

def valueprint(arr, key=None):
    print(key, arr.shape, arr.sum())

print(trf_outputs.keys())


FAIL: layer_h_preln
FAIL: MHAttention_mask
FAIL: MHAttention_mask_scores
dict_keys(['selfAttention_input', 'selfAttention_posEmb', 'selfAttention_mask', 'selfAttention_result', 'selfAttention_result_2', 'layer_x', 'layer_h', 'MHAttention_query', 'MHAttention_key', 'MHAttention_value', 'MHAttention_q', 'MHAttention_k', 'MHAttention_v', 'MHAttention_scores', 'MHAttention_pscores', 'MHAttention_pscores_scores', 'MHAttention_padMaskTile', 'MHAttention_padMaskTile_scores', 'MHAttention_result', 'MHAttention_result_attn', 'MHAttention_result_result', 'MHAttention_result_result_2'])


In [14]:
selfAttention_input = trf_outputs["selfAttention_input"]
selfAttention_posEmb = trf_outputs["selfAttention_posEmb"]
selfAttention_mask = trf_outputs["selfAttention_mask"]

MHAttention_query = trf_outputs["MHAttention_query"]
MHAttention_key = trf_outputs["MHAttention_key"]
MHAttention_value = trf_outputs["MHAttention_value"]
MHAttention_q = trf_outputs["MHAttention_q"]
MHAttention_k = trf_outputs["MHAttention_k"]
MHAttention_v = trf_outputs["MHAttention_v"]
MHAttention_scores = trf_outputs["MHAttention_scores"]
MHAttention_pscores = trf_outputs["MHAttention_pscores"]
MHAttention_pscores_scores = trf_outputs["MHAttention_pscores_scores"]

selfAttention_result = trf_outputs["selfAttention_result"]
selfAttention_result_2 = trf_outputs["selfAttention_result_2"]

valueprint(selfAttention_input, "selfAttention_input")
valueprint(selfAttention_posEmb, "selfAttention_posEmb")
valueprint(selfAttention_mask, "selfAttention_mask")

valueprint(MHAttention_query, "MHAttention_query")
valueprint(MHAttention_key, "MHAttention_key")
valueprint(MHAttention_value, "MHAttention_value")
valueprint(MHAttention_q, "MHAttention_q")
valueprint(MHAttention_k, "MHAttention_k")
valueprint(MHAttention_v, "MHAttention_v")

valueprint(selfAttention_result, "selfAttention_result")
valueprint(selfAttention_result_2, "selfAttention_result_2")

selfAttention_input (1536, 321) 330.57333
selfAttention_posEmb (1839, 384, 4) -1679.4205
selfAttention_mask (321,) 0.0
MHAttention_query (321, 1536) 5019.3354
MHAttention_key (321, 1536) -1448.2941
MHAttention_value (321, 1536) -995.127
MHAttention_q (321, 384, 4) 256.14188
MHAttention_k (321, 384, 4) -1448.2941
MHAttention_v (321, 384, 4) -995.127
selfAttention_result (321, 1536) -1017.8952
selfAttention_result_2 (1536, 321) 25.706522


In [1]:
# testing after fixing the model & weights
import torch
from transformers import MCTCForCTC, MCTCProcessor, MCTCConfig
config = MCTCConfig()
model = MCTCForCTC(config)
model.load_state_dict(torch.load("./ported_pytorch_model.bin"))
model = model.eval()
our_selfattn = model.mctc.encoder.layers[0].attention.self
our_selfattn

MCTCSelfAttention(
  (query): Linear(in_features=1536, out_features=1536, bias=False)
  (key): Linear(in_features=1536, out_features=1536, bias=False)
  (value): Linear(in_features=1536, out_features=1536, bias=False)
  (dropout): Dropout(p=0.3, inplace=False)
  (distance_embedding): Embedding(1839, 384)
)

In [8]:
our_selfattn_output = torch.Tensor(selfAttention_input).unsqueeze(0).transpose(1, 2)
valueprint(our_selfattn_output)
our_selfattn_output = our_selfattn(our_selfattn_output)[0]
valueprint(our_selfattn_output)

None torch.Size([1, 321, 1536]) tensor(330.5734)
query_layer torch.Size([1, 4, 321, 384]) tensor(256.1419, grad_fn=<SumBackward0>)
key_layer torch.Size([1, 4, 321, 384]) tensor(-1448.2943, grad_fn=<SumBackward0>)
value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, grad_fn=<SumBackward0>)
attention_scores torch.Size([1, 4, 321, 321]) tensor(30256.4258, grad_fn=<SumBackward0>)
positional_embedding torch.Size([321, 321, 384]) tensor(101717.8359, grad_fn=<SumBackward0>)
relative_position_scores torch.Size([1, 4, 321, 321]) tensor(57733.4141, grad_fn=<SumBackward0>)
attention_scores torch.Size([1, 4, 321, 321]) tensor(87989.8203, grad_fn=<SumBackward0>)
attention_probs torch.Size([1, 4, 321, 321]) tensor(1283.9999, grad_fn=<SumBackward0>)
None torch.Size([1, 321, 1536]) tensor(-1023.2032, grad_fn=<SumBackward0>)


In [16]:
'''
Up to Q,K,V checks out:
query_layer torch.Size([1, 4, 321, 384]) tensor(256.1419, grad_fn=<SumBackward0>)
key_layer torch.Size([1, 4, 321, 384]) tensor(-1448.2943, grad_fn=<SumBackward0>)
value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, grad_fn=<SumBackward0>)
attention_scores torch.Size([1, 4, 321, 321]) tensor(30256.4258, grad_fn=<SumBackward0>)
'''
valueprint(MHAttention_q, "MHAttention_q")
valueprint(MHAttention_k, "MHAttention_k")
valueprint(MHAttention_v, "MHAttention_v")
valueprint(MHAttention_scores, "MHAttention_scores")

MHAttention_q (321, 384, 4) 256.14188
MHAttention_k (321, 384, 4) -1448.2941
MHAttention_v (321, 384, 4) -995.127
MHAttention_scores (321, 321, 4) 30256.422


In [24]:
'''
Attention scores + position embeddings checks out too (the result does, at least; off by value of 300 though?):
relative_position_scores torch.Size([1, 4, 321, 321]) tensor(57733.4141, grad_fn=<SumBackward0>)
attention_scores torch.Size([1, 4, 321, 321]) tensor(87989.8203, grad_fn=<SumBackward0>)
attention_probs torch.Size([1, 4, 321, 321]) tensor(1283.9999, grad_fn=<SumBackward0>)
None torch.Size([1, 321, 1536]) tensor(-1023.2032, grad_fn=<SumBackward0>)
'''
MHAttention_result_attn = trf_outputs["MHAttention_result_attn"]
MHAttention_result_result_2 = trf_outputs["MHAttention_result_result_2"]
valueprint(MHAttention_pscores, "MHAttention_pscores")
valueprint(MHAttention_pscores_scores, "MHAttention_pscores_scores")
valueprint(MHAttention_result_attn, "MHAttention_result_attn")
valueprint(MHAttention_result_result_2, "MHAttention_result_result_2")

MHAttention_pscores (2159, 321, 4) -85057.44
MHAttention_pscores_scores (321, 321, 4) 87683.41
MHAttention_result_attn (321, 321, 4) 1283.9999
MHAttention_result_result_2 (321, 1536) -1017.8952


In [25]:
valueprint(our_selfattn_output, "our_selfattn_output")
valueprint(MHAttention_result_result_2, "MHAttention_result_result_2")

our_selfattn_output torch.Size([1, 321, 1536]) tensor(-1023.2032, grad_fn=<SumBackward0>)
MHAttention_result_result_2 (321, 1536) -1017.8952


In [28]:
selfAttention_result = trf_outputs["selfAttention_result"]
valueprint(selfAttention_result, "selfAttention_result")

selfAttention_result (321, 1536) -1017.8952


In [32]:
'''
arr = result.array();
af::saveArray("selfAttention_result", arr, savePathChar, true); 

result = (*wf_)(transpose(result));

arr = result.array();
af::saveArray("selfAttention_result_2", arr, savePathChar, true); 

return result;
'''

selfAttention_result_2 = trf_outputs["selfAttention_result_2"]
valueprint(selfAttention_result_2, "selfAttention_result_2")


selfAttention_result_2 (1536, 321) 25.706522


In [33]:
our_attn = model.mctc.encoder.layers[0].attention
our_attn

MCTCAttention(
  (self): MCTCSelfAttention(
    (query): Linear(in_features=1536, out_features=1536, bias=False)
    (key): Linear(in_features=1536, out_features=1536, bias=False)
    (value): Linear(in_features=1536, out_features=1536, bias=False)
    (dropout): Dropout(p=0.3, inplace=False)
    (distance_embedding): Embedding(1839, 384)
  )
  (output): MCTCSelfOutput(
    (dense): Linear(in_features=1536, out_features=1536, bias=False)
    (LayerNorm): MCTCLayerNorm()
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

In [34]:
our_attn_output = torch.Tensor(selfAttention_input).unsqueeze(0).transpose(1, 2)
valueprint(our_attn_output)
our_attn_output = our_attn(our_attn_output)[0]
valueprint(our_attn_output)

None torch.Size([1, 321, 1536]) tensor(330.5734)
query_layer torch.Size([1, 4, 321, 384]) tensor(256.1419, grad_fn=<SumBackward0>)
key_layer torch.Size([1, 4, 321, 384]) tensor(-1448.2943, grad_fn=<SumBackward0>)
value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, grad_fn=<SumBackward0>)
attention_scores torch.Size([1, 4, 321, 321]) tensor(30256.4258, grad_fn=<SumBackward0>)
positional_embedding torch.Size([321, 321, 384]) tensor(101717.8359, grad_fn=<SumBackward0>)
relative_position_scores torch.Size([1, 4, 321, 321]) tensor(57733.4141, grad_fn=<SumBackward0>)
attention_scores torch.Size([1, 4, 321, 321]) tensor(87989.8203, grad_fn=<SumBackward0>)
attention_probs torch.Size([1, 4, 321, 321]) tensor(1283.9999, grad_fn=<SumBackward0>)
None torch.Size([1, 321, 1536]) tensor(-618.0341, grad_fn=<SumBackward0>)


In [1]:
# testing again after chaning MCTCSelfOutput to this differnt way flashlight seems to do things

# testing after fixing the model & weights
import torch
from transformers import MCTCForCTC, MCTCProcessor, MCTCConfig
config = MCTCConfig()
model = MCTCForCTC(config)
model.load_state_dict(torch.load("./ported_pytorch_model.bin"))
model = model.eval()
our_attn = model.mctc.encoder.layers[0].attention


In [6]:
# lets take a deeper look.
import arrayfire as af

possib_keys = ["selfAttention_input", "selfAttention_posEmb", "selfAttention_mask", "selfAttention_result", "selfAttention_result_2",
"layer_x", "layer_h_preln", "layer_h", "MHAttention_query", "MHAttention_key", "MHAttention_value", "MHAttention_q", "MHAttention_k", "MHAttention_v",
"MHAttention_scores", "MHAttention_pscores", "MHAttention_pscores_scores", "MHAttention_mask", "MHAttention_mask_scores", "MHAttention_padMaskTile", "MHAttention_padMaskTile_scores",
"MHAttention_result", "MHAttention_result_attn", "MHAttention_result_result", "MHAttention_result_result_2"]


selfAttention_input = af.array.read_array("OUTPUT_TRF.arr", key="selfAttention_input").to_ndarray()
MHAttention_result_result = af.array.read_array("OUTPUT_TRF.arr", key="MHAttention_result_result").to_ndarray()
MHAttention_result_result_2 = af.array.read_array("OUTPUT_TRF.arr", key="MHAttention_result_result_2").to_ndarray()
selfAttention_result_2 = af.array.read_array("OUTPUT_TRF.arr", key="selfAttention_result_2").to_ndarray()
layer_x = af.array.read_array("OUTPUT_TRF.arr", key="layer_x").to_ndarray()
layer_h = af.array.read_array("OUTPUT_TRF.arr", key="layer_h").to_ndarray()
        


In [3]:
def valueprint(arr, key=None):
    print(key, arr.shape, arr.sum())
our_attn_output = torch.Tensor(selfAttention_input).unsqueeze(0).transpose(1, 2)
valueprint(our_attn_output)
our_attn_output = our_attn(our_attn_output)[0]
valueprint(our_attn_output)

None torch.Size([1, 321, 1536]) tensor(330.5734)
SelfOUtput input torch.Size([1, 321, 1536]) tensor(-1023.2032, grad_fn=<SumBackward0>)
SelfOUtput prenorm torch.Size([1, 321, 1536]) tensor(22.9749, grad_fn=<SumBackward0>)
SelfOUtput input_tensor torch.Size([1, 321, 1536]) tensor(330.5734)
SelfOUtput postnorm torch.Size([1, 321, 1536]) tensor(-618.0341, grad_fn=<SumBackward0>)
None torch.Size([1, 321, 1536]) tensor(-618.0341, grad_fn=<SumBackward0>)


In [4]:
'''
SelfOUtput prenorm torch.Size([1, 321, 1536]) tensor(22.9749, grad_fn=<SumBackward0>)
'''
valueprint(selfAttention_result_2, "selfAttention_result_2")

selfAttention_result_2 (1536, 321) 25.706522


In [5]:
''' SUPPOSED TO BE THE SAME!

auto h = (*norm1_)((f * selfAttention(input)).as(x.type()) + x);
af::saveArray("layer_h", h, savePathChar, true); 

WHERE
selfAttention_result_2 = selfAttention(input)
f = 1.0

SO:
h = LayerNorm(selfAttention_result_2 + x)

AND:
print("SelfOUtput prenorm", hidden_states.shape, hidden_states.sum())

hidden_states = self.LayerNorm(hidden_states + input_tensor)
print("SelfOUtput postnorm", hidden_states.shape, hidden_states.sum())
'''
valueprint(layer_h, "layer_h")

layer_h (1536, 321) -1035.2311


In [8]:
'''[YES] x in fl == input_tensor?
SelfOUtput input_tensor torch.Size([1, 321, 1536]) tensor(330.5734)
'''
valueprint(layer_x, "layer_x")

layer_x (1536, 321) 330.57333
